In [1]:
import requests
import pandas as pd

base_item_url = "https://poe.ninja/api/data/itemoverview?league={league}&type={item}"
base_currency_url = "https://poe.ninja/api/data/currencyoverview?league={league}&type={item}"
league = "Ancestor"

In [2]:
def template_replace(template_input, item_type):
    '''
    Replace template url to be an f-string
    '''
    item = item_type
    return eval(f"f'{template_input}'")

In [3]:
def chance_calculator(prob_occurence):
    fail_prob = 1 - prob_occurence
    tot_chance = fail_prob
    times = 1
    while tot_chance >= prob_occurence:
        tot_chance = fail_prob ** times
        times += 1
    return times

In [4]:
def get_currency_data():
    currencies = ["Currency"]
    currency_price_data = {}
    raw_currency_data = {}

    for currency in currencies:
        raw_currency_data[currency] = requests.get(template_replace(base_currency_url, currency)).json()
        
    for currency in raw_currency_data["Currency"]["lines"]:
        currency_price_data[currency["currencyTypeName"]] = {
           "chaosValue": currency["chaosEquivalent"]
        }
    
    return pd.DataFrame.from_dict(currency_price_data, orient="index")

In [5]:
def get_item_data():
    items = ["Scarab", "DeliriumOrb", "Essence"]
    item_price_data = {}
    raw_item_data = {}

    for item in items:
        raw_item_data[item] = requests.get(template_replace(base_item_url, item)).json()
    
    for category, data in raw_item_data.items():
        for item in data.get("lines"):
            item_price_data[item["name"]] = {
                "chaosValue": item["chaosValue"],
                "exaltedValue": item["exaltedValue"],
                "divineValue": item["divineValue"]
            }

    return pd.DataFrame.from_dict(item_price_data, orient="index")

In [6]:
currency_df = get_currency_data()

In [7]:
item_df = get_item_data()

In [8]:
harvest_df = currency_df.filter(like='Lifeforce', axis=0)
delirium_df = item_df.filter(like='Delirium Orb', axis=0)
scarab_df = item_df.filter(like='Scarab', axis=0)
essence_df = item_df.filter(like='Essence', axis=0)

In [9]:
max_value_delirium = delirium_df["chaosValue"].max()

In [10]:
primal_lifeforce_unit_price = harvest_df.at["Primal Crystallised Lifeforce", 'chaosValue']
vivid_lifeforce_unit_price = harvest_df.at["Vivid Crystallised Lifeforce", 'chaosValue']
wild_lifeforce_unit_price = harvest_df.at["Wild Crystallised Lifeforce", 'chaosValue']

In [64]:
def fixed_weight_profit_calc(item_df, item_tiers, stack_size, min_profit_margin, craft_cost):   
    for tier in item_tiers:
        calc_df = item_df.filter(like=tier, axis=0)
        
        min_value_item = calc_df["chaosValue"].min()
        
        base_cost_item = min_value_item * stack_size
        craft_try_cost = craft_cost * stack_size
        min_craft_cost = min_value_item * stack_size + craft_try_cost
        
        profit_items = calc_df[calc_df["chaosValue"] * stack_size >= min_craft_cost + min_profit_margin]
        
        if len(profit_items) == 0:
            print(f"No profit on {tier} items :(")
        else:
            for i in range(len(profit_items), 0, -1):
                
                chance_to_hit = len(profit_items.head(i)) / (len(calc_df) - 1)
                worst_case_tries = chance_calculator(chance_to_hit)

                best_scenario = base_cost_item + craft_try_cost * 1
                avg_scenario = base_cost_item + craft_try_cost * (1 / chance_to_hit)
                worst_scenario = base_cost_item + craft_try_cost * worst_case_tries
                
                scenarios = {
                    "Best Scenario": {
                        "cost": best_scenario,
                        "tries": 1
                    },
                    "Avg Scenario": {
                        "cost": avg_scenario,
                        "tries": 1 / chance_to_hit
                    }, 
                    "Worst Scenario": {
                        "cost": worst_scenario,
                        "tries": worst_case_tries
                    }
                }
                
                avg_profit = profit_items.head(i)["chaosValue"].sum() * 10 / len(profit_items.head(i))

                print(f"Calculating profit when targetting {profit_items.head(i).index.tolist()}")

                for scenario, info in scenarios.items():
                    cost = info["cost"]
                    tries = info["tries"]

                    profit = avg_profit - cost
                    print(f"""{scenario} cost: {cost:.0f} C. 
                        Base item: {min_value_item} C x {stack_size} = {min_value_item * stack_size} C.
                        Crafting try: {craft_try_cost} C x {tries} = {craft_try_cost * tries} C.
                        Profit: {int(profit)} C!\n""")
                print("\n")



In [65]:
fixed_weight_profit_calc(delirium_df, ["Delirium Orb"], 10, 20, 30 * primal_lifeforce_unit_price)

Calculating profit when targetting ['Skittering Delirium Orb', "Diviner's Delirium Orb", 'Fine Delirium Orb']
Best Scenario cost: 89 C. 
                        Base item: 8.0 C x 10 = 80.0 C.
                        Crafting try: 9.0 C x 1 = 9.0 C.
                        Profit: 125 C!

Avg Scenario cost: 134 C. 
                        Base item: 8.0 C x 10 = 80.0 C.
                        Crafting try: 9.0 C x 6.0 = 54.0 C.
                        Profit: 80 C!

Worst Scenario cost: 179 C. 
                        Base item: 8.0 C x 10 = 80.0 C.
                        Crafting try: 9.0 C x 11 = 99.0 C.
                        Profit: 35 C!



Calculating profit when targetting ['Skittering Delirium Orb', "Diviner's Delirium Orb"]
Best Scenario cost: 89 C. 
                        Base item: 8.0 C x 10 = 80.0 C.
                        Crafting try: 9.0 C x 1 = 9.0 C.
                        Profit: 177 C!

Avg Scenario cost: 161 C. 
                        Base item: 8.0 C x 10 =

In [66]:
fixed_weight_profit_calc(scarab_df, ["Rusted", "Polished", "Gilded", "Winged"], 10, 20, 30 * wild_lifeforce_unit_price)

No profit on Rusted items :(
Calculating profit when targetting ['Polished Divination Scarab', 'Polished Legion Scarab', 'Polished Ambush Scarab']
Best Scenario cost: 19 C. 
                        Base item: 1.0 C x 10 = 10.0 C.
                        Crafting try: 9.0 C x 1 = 9.0 C.
                        Profit: 54 C!

Avg Scenario cost: 55 C. 
                        Base item: 1.0 C x 10 = 10.0 C.
                        Crafting try: 9.0 C x 5.0 = 45.0 C.
                        Profit: 18 C!

Worst Scenario cost: 91 C. 
                        Base item: 1.0 C x 10 = 10.0 C.
                        Crafting try: 9.0 C x 9 = 81.0 C.
                        Profit: -17 C!



Calculating profit when targetting ['Polished Divination Scarab', 'Polished Legion Scarab']
Best Scenario cost: 19 C. 
                        Base item: 1.0 C x 10 = 10.0 C.
                        Crafting try: 9.0 C x 1 = 9.0 C.
                        Profit: 71 C!

Avg Scenario cost: 78 C. 
            